<a href="https://colab.research.google.com/github/franfasan1/DH_G8_desafio3/blob/main/notebook_desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>